In [4]:
import pandas as pd
import numpy as np

import plotly.plotly as py
import plotly.graph_objs as go
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot

#suppress warnings
import warnings
warnings.simplefilter("ignore")


In [5]:
df=pd.read_stata('ps1.dta')

In [6]:
df['employed']=np.where(df['work']==1,1,0)
df['unemployed']=np.where(df['work']==0,1,0)
df['parent']=np.where(df['children']!=0,1,0)

#pivot by year and parent and then reset the index
df1=df.groupby(['year', 'parent']).sum()
df1=df1.reset_index()

#calculate the lfpr for both parents and no parents
df1['urate']=(df1['employed'])/(df1['employed']+df1['unemployed'])
parent=df1[df1['parent']==1]
nparent=df1[df1['parent']==0]

In [7]:
# Add data
year = parent['year']
parentLMPR= parent['urate']
nparentLMPR = nparent['urate']


# Create and style traces
trace0 = go.Scatter(
    x = year,
    y = parentLMPR,
    name = 'W/ Children',
    line = dict(
        color = ('blue'),
        width = 2)
)
trace1 = go.Scatter(
    x = year,
    y = nparentLMPR,
    name = 'W/O Children',
    line = dict(
        color = ('red'),
        width = 2,)
)


data = [trace0, trace1]

# Edit the layout
layout = dict(title = 'Single Women Labor Market Participation Rates',
              xaxis = dict(title = 'Year'),
              yaxis = dict(title = 'LMPR'),
              )

fig = dict(data=data, layout=layout)
py.iplot(fig, filename='raw-plot')

### Reindex the LMPR to 1 in 1991

In [8]:
pBaseLevel=parent.iloc[0,3]
nBaseLevel=nparent.iloc[0,3]
parent['index']=parent['urate']/pBaseLevel
nparent['index']=nparent['urate']/nBaseLevel

nBaseLevel

0.5830324909747292

In [9]:

# Add data
year = parent['year']
piLMPR= parent['index']
niLMPR = nparent['index']


# Create and style traces
trace0 = go.Scatter(
    x = year,
    y = piLMPR,
    name = 'W/ Children',
    line = dict(
        color = ('blue'),
        width = 2)
)
trace1 = go.Scatter(
    x = year,
    y = niLMPR,
    name = 'W/O Children',
    line = dict(
        color = ('red'),
        width = 2,)
)


data = [trace0, trace1]

# Edit the layout
layout = dict(title = 'Single Women Labor Market Participation Rates, Indexed to 1991 rates',
              xaxis = dict(title = 'Year'),
              yaxis = dict(title = 'LMPR'),
              )

fig = dict(data=data, layout=layout)
py.iplot(fig, filename='index-plot')

TODO:  Insert comments on validity of using single women with children as a control group.

### In this section I calculate the difference in differences for table two.  I start back at the original dataframe to get clean averages.

In [10]:

parent=df[df['parent']==1]
nparent=df[df['parent']!=1]

#calculate the average of the treatment group pre-1994
tc1=parent[parent['year']<1994]
tc1_empl=tc1['work'].sum()
tc1_mean=tc1_empl/len(tc1)

#calculate the average of the treatment group post-1994
tc2=parent[parent['year']>1993]
tc2_empl=tc2['work'].sum()
tc2_mean=tc2_empl/len(tc1)

#calculate the average of the control group pre-1994
cg1=nparent[nparent['year']<1994]
cg1_empl=cg1['work'].sum()
cg1_mean=cg1_empl/len(cg1)

#calculate the average of the control group post-1994
cg2=nparent[nparent['year']>1993]
cg2_empl=cg2['work'].sum()
cg2_mean=cg2_empl/len(tc1)

#calculate diffs
dif1=tc2_mean-tc1_mean
dif2=cg2_mean-cg1_mean
dif_dif=dif1-dif2

#print (tc1_mean, tc2_mean, cg1_mean, cg2_mean)

l1=["Treatment Group", len(parent), tc1_mean, tc2_mean, dif1, '']
l2=["Control Group", len(nparent), cg1_mean, cg2_mean, dif2, dif_dif]

table=[l1, l2]

headers=['Group', 'Sample Size', 'Pre-1993', 'Post-1993', 'Difference', 'Difference-in-differences']

table2=pd.DataFrame(table, columns=headers)

table2


Group  Sample Size  Pre-1993  Post-1993  Difference  \
0  Treatment Group         7819  0.445962   0.412762   -0.033200   
1    Control Group         5927  0.575460   0.374382   -0.201078   

  Difference-in-differences  
0                            
1                  0.167878

### TODO:  Comment on this table

### Now we do the same comparison except for women with one child and women with two or more children.

In [11]:
df

state    year  urate  children  nonwhite           finc           earn  \
0       11.0  1991.0    7.6         3         0    7970.792951    7970.792951   
1       11.0  1991.0    7.6         0         0   31227.973568   14730.176211   
2       11.0  1991.0    7.6         0         0    6143.072687     589.207048   
3       11.0  1991.0    7.6         0         0   16761.762115       0.000000   
4       11.0  1991.0    7.6         1         1    7424.008811    2121.145374   
5       11.0  1991.0    7.6         0         0    8248.898678    8248.898678   
6       11.0  1991.0    7.6         0         0    7183.612335       0.000000   
7       11.0  1991.0    7.6         2         0    6405.859031       0.000000   
8       11.0  1991.0    7.6         0         1   18714.394273   18714.394273   
9       11.0  1991.0    7.6         0         0   17482.951542       0.000000   
10      11.0  1991.0    7.6         0         0   32382.819383   31581.497797   
11      11.0  1991.0    7.6         3         0    6405.859031       0.000000   
12      11.0  1991.0    7.6         3         0    8046.211454       0.000000   
13      11.0  1991.0    7.6         1         0    7499.427313       0.000000   
14      11.0  1991.0    7.6         0         0    9630.000000       0.000000   
15      11.0  1991.0    7.6         1         0    4765.506608       0.000000   
16      11.0  1991.0    7.6         0         0   54330.781938   17676.211454   
17      11.0  1991.0    7.6         1         0   20452.555066       0.000000   
18      11.0  1992.0    7.2         0         0   21637.184604   17159.657876   
19      11.0  1992.0    7.2         0         0   11603.360656   10867.783321   
20      11.0  1992.0    7.2         2         0    4499.262295    1029.579473   
21      11.0  1992.0    7.2         1         0   19572.305773   17159.657876   
22      11.0  1992.0    7.2         2         0       0.000000       0.000000   
23      11.0  1992.0    7.2         0         0   21180.737705   19447.612259   
24      11.0  1992.0    7.2         2         0    6831.831789       0.000000   
25      11.0  1992.0    7.2         0         0    4575.908767       0.000000   
26      11.0  1992.0    7.2         1         0   19572.305773   17159.657876   
27      11.0  1992.0    7.2         3         0    8685.074840       0.000000   
28      11.0  1992.0    7.2         0         0   19459.052031   19447.612259   
29      11.0  1992.0    7.2         1         0    9292.526728       0.000000   
...      ...     ...    ...       ...       ...            ...            ...   
13716   95.0  1993.0    4.3         3         0       0.000000       0.000000   
13717   95.0  1993.0    4.3         2         0  115884.332180  108766.795850   
13718   95.0  1994.0    6.1         2         0  120406.406880  114364.372470   
13719   95.0  1994.0    6.1         0         0   47857.591093   47857.591093   
13720   95.0  1994.0    6.1         7         0   23003.917004   12542.176113   
13721   95.0  1994.0    6.1         0         0   16439.878543   16244.939271   
13722   95.0  1994.0    6.1         2         0    4686.123482       0.000000   
13723   95.0  1994.0    6.1         3         0    7473.755061       0.000000   
13724   95.0  1994.0    6.1         2         0   18506.234818       0.000000   
13725   95.0  1994.0    6.1         3         0   11163.522267       0.000000   
13726   95.0  1994.0    6.1         0         0    7667.611336    7667.611336   
13727   95.0  1994.0    6.1         0         0   17399.412955       0.000000   
13728   95.0  1995.0    5.9         1         0    7013.976378       0.000000   
13729   95.0  1995.0    5.9         0         0   11879.527559   11584.645669   
13730   95.0  1995.0    5.9         0         0   21655.915354   15270.669291   
13731   95.0  1995.0    5.9         1         0   19024.094488       0.000000   
13732   95.0  1995.0    5.9         0         0   28252.844488   28179.124016   
13733   95.0  1995.0    5.9         0 

In [16]:
one_child=df[df['children']==1]
two_child=df[df['children']>1]

#calculate the average of the treatment group with one child pre-1994
tg1c1=one_child[one_child['year']<1994]
tg1c1_empl=tg1c1['work'].sum()
tg1c1_mean=tg1c1_empl/len(tg1c1)

#calculate the average of the treatment group with one child post-1994
tg2c1=one_child[one_child['year']>1993]
tg2c1_empl=tg2c1['work'].sum()
tg2c1_mean=tg2c1_empl/len(tg2c1)

#calculate the average of the treatment group with two children pre-1994
tg1c2=two_child[two_child['year']<1994]
tg1c2_empl=tg1c2['work'].sum()
tg1c2_mean=tg1c2_empl/len(tg1c2)

#calculate the average of the treatment group with two child post-1994
tg2c2=two_child[two_child['year']>1993]
tg2c2_empl=tg2c2['work'].sum()
tg2c2_mean=tg2c2_empl/len(tg2c2)

#calculate diffs
dif3=tg1c2_mean-tg1c1_mean
dif4=tg2c2_mean-tg1c2_mean
dif_dif3=dif3-dif2
dif_dif4=dif4-dif2

l3=["One Child", len(one_child), tg1c1_mean, tg2c1_mean, dif3, '']
l4=["Control Group", len(nparent), cg1_mean, cg2_mean, dif2, dif_dif3]

l5=["Two Child", len(two_child), tg1c2_mean, tg2c2_mean, dif4, '']
l6=["Control Group", len(nparent), cg1_mean, cg2_mean, dif2, dif_dif4]

table=[l1, l2, l3, l4, l5, l6]

headers=['Group', 'Sample Size', 'Pre-1993', 'Post-1993', 'Difference', 'Difference-in-differences']

table2=pd.DataFrame(table, columns=headers)

table2



Group  Sample Size  Pre-1993  Post-1993  Difference  \
0  Treatment Group         7819  0.445962   0.412762   -0.033200   
1    Control Group         5927  0.575460   0.374382   -0.201078   
2        One Child         3058  0.523579   0.554131   -0.127127   
3    Control Group         5927  0.575460   0.374382   -0.201078   
4        Two Child         4761  0.396452   0.449723    0.053271   
5    Control Group         5927  0.575460   0.374382   -0.201078   

  Difference-in-differences  
0                            
1                  0.167878  
2                            
3                 0.0739506  
4                            
5                  0.254349